In [1]:
import h5py
import numpy as np
import os

In [2]:
import requests

baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"e3fa769d419027b4bb72798214dfe554"}

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    
    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [3]:
mass_max = 0.0005
mass_star = 0.0
search_query = "?primary_flag=1" + "&mass_stars__lte=" + str(mass_star) + "&mass__lte=" + str(mass_max)

In [4]:
url = "http://www.tng-project.org/api/TNG50-1/snapshots/0/subhalos/" + search_query
# url = "http://www.tng-project.org/api/TNG50-1/snapshots/4/subhalos/46/info.json"
r = get(url)

In [5]:
url

'http://www.tng-project.org/api/TNG50-1/snapshots/0/subhalos/?primary_flag=1&mass_stars__lte=0.0&mass__lte=0.0005'

In [6]:
sub = get( r['results'][0]['url'] )
sub['related']

{'sublink_progenitor': None,
 'sublink_descendant': 'http://www.tng-project.org/api/TNG50-1/snapshots/1/subhalos/27703/',
 'parent_halo': 'http://www.tng-project.org/api/TNG50-1/snapshots/0/halos/15767/'}

In [17]:
# Record the mass of the subhalo
mass_list = []

In [18]:
# If the progenitor subhalo's mass doesn't exceed 10 times the current subhalo's mass

id_list = []
j = 1
for i in range(0, 121):
    current_id = r['results'][i]['id']
    # print(current_id)

    sub = get( r['results'][i]['url'] )
    sub_mass = sub['mass_log_msun']

    des = get( sub['related']['sublink_descendant'] )
    des_mass = des['mass_log_msun']

    if ( des_mass - sub_mass ) < 1.0:
        id_list.append( current_id )
        print("\t", current_id)
        mass_list.append( sub_mass )
        j += 1

    if (j >= 30):
        break

	 26221
	 26616
	 26842
	 27062
	 27075
	 27261
	 27836
	 27856
	 28199
	 28323
	 28336
	 28585
	 28904
	 29124
	 29773
	 30037
	 30112
	 30251
	 30269
	 30331
	 30476
	 30779
	 31111
	 31393
	 31445
	 31590
	 31708
	 31812
	 31859


In [21]:
mass_list.append(6.836607760581343)
mass_list.append(6.85129988406522)

In [22]:
10 ** np.max(mass_list), 10 ** np.min(mass_list)

(7366400.944788899, 6495423.678771777)

In [10]:
# Download the progenitor and the current subhalos' hdf5

ini_num = 0

for i, id in enumerate(id_list):
    url = "http://www.tng-project.org/api/TNG50-1/snapshots/0/subhalos/"

    params = {'gas': 'all', 'dm':'all' }

    sub = get( url + str(id) )
    des = get( sub['related']['sublink_descendant'] )

    des_id = sub['desc_sfid']
    pro_id = des['prog_sfid']

    sub_saved_filename = get( sub['cutouts']['parent_halo'], params )

    os.system('mkdir %02d_subhalo' % (ini_num + i) + str(id))

    if id == pro_id:    # Print id of the subhalo which has no second progenitor.
        print( id, 1 )
        os.system('mv ' + sub_saved_filename  + ' %02d_subhalo' % (ini_num + i) + str(id) + '_r')
    else:               # Download the progenitor and the current subhalos
        print( id, 2 )
        pro = get( url + str(pro_id) )
        pro_saved_filename = get( pro['cutouts']['parent_halo'], params )

        os.system('mv ' + sub_saved_filename + ' ' + pro_saved_filename  
                  + ' %02d_subhalo' % (ini_num + i) + str(id))


In [4]:
# desc = get( sub['related']['sublink_descendant'] )
url = "http://www.tng-project.org/api/TNG50-1/snapshots/0/subhalos/7181/"
sub = get( url )
get( sub['trees']['sublink_mpb'])

'sublink_mpb_7181.hdf5'

In [14]:
prev = 0
url += "&offset=" + str( prev*100 )
r = get( url )

In [ ]:
record = []
for i in range( prev, (r['count'] // 100) + 1 ):
    # for j in range( min( 100, r['count'] - i*100 )):
    for j in range( 100 ):
        if ( 100 * i + j ) == r['count']:
            break

        url = r['results'][j]['url']
        sub = get( url )

        if sub['related']['sublink_descendant'] != None:
            # print( "id with pro:", sub['id'] )
            parent_halo = sub['related']['parent_halo']
            record.append( parent_halo.split('/')[-2] )

            # record the mass of subhalo
            sub_mass = 
    
    if i < r['count'] // 100:
        r = get( r['next'] )
    print("progress:", i*100)

In [27]:
candiate = {}
for i in record:
    halo = get( 'http://www.tng-project.org/api/TNG50-1/snapshots/0/halos/' + i )
    if halo["child_subhalos"]["count"] > 1:
        candiate[int(i)] = halo["child_subhalos"]["count"]

candiate

In [21]:
r['results'][0]

{'id': 16130,
 'sfr': 0.0,
 'mass_log_msun': 6.836607760581343,
 'url': 'http://www.tng-project.org/api/TNG50-1/snapshots/0/subhalos/16130/'}

In [52]:
f = h5py.File( "sublink_mpb_16130.hdf5" )
f = h5py.File( "sublink_mpb_27703.hdf5" )
# f = h5py.File( "sublink_60.hdf5" )
# f = h5py.File( "sublink_641.hdf5" )

In [50]:
f.keys()

<KeysViewHDF5 ['DescendantID', 'FirstProgenitorID', 'FirstSubhaloInFOFGroupID', 'GroupBHMass', 'GroupBHMdot', 'GroupCM', 'GroupFirstSub', 'GroupGasMetalFractions', 'GroupGasMetallicity', 'GroupLen', 'GroupLenType', 'GroupMass', 'GroupMassType', 'GroupNsubs', 'GroupPos', 'GroupSFR', 'GroupStarMetalFractions', 'GroupStarMetallicity', 'GroupVel', 'GroupWindMass', 'Group_M_Crit200', 'Group_M_Crit500', 'Group_M_Mean200', 'Group_M_TopHat200', 'Group_R_Crit200', 'Group_R_Crit500', 'Group_R_Mean200', 'Group_R_TopHat200', 'LastProgenitorID', 'MainLeafProgenitorID', 'Mass', 'MassHistory', 'NextProgenitorID', 'NextSubhaloInFOFGroupID', 'NumParticles', 'RootDescendantID', 'SnapNum', 'SubfindID', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGasMetalFractions', 'SubhaloGasMetalFractionsHalfRad', 'SubhaloGasMetalFractionsMaxRad', 'SubhaloGasMetalFractionsSfr', 'SubhaloGasMetalFractionsSfrWeighted', 'SubhaloGasMetallicity', 'SubhaloGasMetallicityHalfRad', 'SubhaloGasMetallicityMaxRad', 'Subh

In [6]:
f = h5py.File( "sublink_mpb_16130.hdf5" )
flag = ['DescendantID', 'SubhaloID','FirstProgenitorID', 'NextProgenitorID','LastProgenitorID', 'MainLeafProgenitorID', 'SubfindID', 'SnapNum']
for i in flag:
    print(i)
    print( f[i][:] )

DescendantID
[67301294]
SubhaloID
[67301296]
FirstProgenitorID
[-1]
NextProgenitorID
[-1]
LastProgenitorID
[67301296]
MainLeafProgenitorID
[67301296]
SubfindID
[16130]
SnapNum
[0]


In [7]:
f = h5py.File( "sublink_mpb_7181.hdf5" )
flag = ['DescendantID', 'SubhaloID','FirstProgenitorID', 'NextProgenitorID','LastProgenitorID', 'SubfindID', 'SnapNum']
for i in flag:
    print(i)
    print( f[i][:] )

DescendantID
[67301294]
SubhaloID
[67301295]
FirstProgenitorID
[-1]
NextProgenitorID
[67301296]
LastProgenitorID
[67301295]
MainLeafProgenitorID
[67301295]
SubfindID
[7181]
SnapNum
[0]


In [8]:
f = h5py.File( "sublink_mpb_27703.hdf5" )
flag = ['DescendantID', 'SubhaloID','FirstProgenitorID', 'NextProgenitorID','LastProgenitorID', 'SubfindID', 'SnapNum']
for i in flag:
    print(i)
    print( f[i][:] )

DescendantID
[67301293 67301294]
SubhaloID
[67301294 67301295]
FirstProgenitorID
[67301295       -1]
NextProgenitorID
[67301297 67301296]
LastProgenitorID
[67301296 67301295]
MainLeafProgenitorID
[67301295 67301295]
SubfindID
[27703  7181]
SnapNum
[1 0]


In [23]:
ids = [46, 60, 62]
for id in ids:
    url = "http://www.tng-project.org/api/TNG50-1/snapshots/4/subhalos/" + str(id)
    sub = get( url )
    saved_filename = get( sub['cutouts']['subhalo'] )
    print( id, saved_filename )

46 cutout_46.hdf5
60 cutout_60.hdf5
62 cutout_62.hdf5
